Предложили варианты.
Начну я с первого боевого подхода - fine-tuning (T5 от Google, RuBert, GPT)

Подходы:
1. Поиск токенов в вопросе - бейзлайн. 
2. Дальше подумать, как можно научить модель отвечать на вопросы без контекста
- Гуглинг (строим базу знаний, модель выбирает наиболее релевантный текст, а дальше накатываем модель 1)
- Менее оптимизирвоанный вариант ест весь здоровенный текст в качестве контекста
        - 

### Поиск токенов ответа

##### Описание решения
В таргете даны позиция начала и конца ответа, попробуем научить модель выделять эти позиции. 
Сведем к задаче классификации. Для каждого токена модель будет предсказывать вероятность начала ответа в нем и вероятности конца ответа в нем.
Модель выдает вектор вероятностей размерности длины последовательности. Далее нужно замаскировать позиции, соответствующие токенам вопроса и служебным токенам, после чего прогнать через софтмакс.


##### Обучение
Пока идея - приделать две независимых головы для начала и для конца и учить модель на сумму их лоссов


##### Инференс
Вычислим веротяности всех пар, где start_pos <= end_pos, а из них выберем самую вероятную (хотя считать веротяности независимыми странно)
Можно загрузить в бота текст для поиска ответа отдельной командой, а далее задавать к тексту вопросы.

А может стоит и ответ к вопросу присобачить.

Может ли модель сильно полагаться на то, что после вопроса есть токен [SEP]?


In [3]:
import subprocess
import matplotlib.pyplot as plt
import numpy as np
import datasets
from datasets import load_dataset
from tqdm import tqdm
from tokenizers import Tokenizer
from transformers import BertTokenizer, BertTokenizerFast

In [4]:
ds = load_dataset("kuznetsoffandrey/sberquad")

In [5]:
train_set = ds["train"]
valid_set = ds["validation"]
test_set = ds["test"]

print("Context: ", train_set[0]["context"])
print("Question: ", train_set[0]["question"])
print("Answer: ", train_set[0]["answers"])

Context:  В протерозойских отложениях органические остатки встречаются намного чаще, чем в архейских. Они представлены известковыми выделениями сине-зелёных водорослей, ходами червей, остатками кишечнополостных. Кроме известковых водорослей, к числу древнейших растительных остатков относятся скопления графито-углистого вещества, образовавшегося в результате разложения Corycium enigmaticum. В кремнистых сланцах железорудной формации Канады найдены нитевидные водоросли, грибные нити и формы, близкие современным кокколитофоридам. В железистых кварцитах Северной Америки и Сибири обнаружены железистые продукты жизнедеятельности бактерий.
Question:  чем представлены органические остатки?
Answer:  {'text': ['известковыми выделениями сине-зелёных водорослей'], 'answer_start': [109]}


Проверим, что ответы для всех датасетов определены однозначно

In [6]:
assert len(train_set.filter(lambda x: len(x["answers"]["text"]) != 1)) == 0
assert len(valid_set.filter(lambda x: len(x["answers"]["text"]) != 1)) == 0
assert len(test_set.filter(lambda x: len(x["answers"]["text"]) != 1)) == 0


Начнем с модели rubert-base-cased (токенизирует все слова с учетом регистра)
1. Обучался на русской википедии
2. 12 слоев энкодера, 12 голов аттеншена
3. В качестве tokenizer использовался WordPiece на 30к токенов в словаре

In [7]:
MAX_LENGTH = 256
DOC_STRIDE = 64
BATCH_SIZE = 128
NUM_EPOCHS = 3
MODEL_CHECKPOINT = "cointegrated/rubert-tiny" #"DeepPavlov/rubert-base-cased"

Нужно получить те же токены, которые использовались при обучении берта.

clean_up_tokenization_spaces  — должна ли модель очищать пробелы, которые были добавлены при разделении входного текста в процессе токенизации.
do_lower_case - преобразовывает все токены в нижний регистр при токенизации (а что происходит, если в словаре они в верхнем регистре?)

In [8]:
subprocess.run(["powershell", "-Command", f"wget https://huggingface.co/{MODEL_CHECKPOINT}/resolve/main/vocab.txt -OutFile vocab.txt"], capture_output=True)

CompletedProcess(args=['powershell', '-Command', 'wget https://huggingface.co/cointegrated/rubert-tiny/resolve/main/vocab.txt -OutFile vocab.txt'], returncode=0, stdout=b'', stderr=b'')

In [9]:
tokenizer = BertTokenizerFast("vocab.txt", do_lower_case=False, clean_up_tokenization_spaces=True, padding_side="right")
tokenizer

BertTokenizerFast(name_or_path='', vocab_size=29564, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	4: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

- Мы можем обрезать тексты, но тогда ответ можем не попасть к вопросу. Разобьем текст на k пересекающихся окон.
- Если ответа там нет или он обрезан, положим start=end и пустую строку в качестве ответа. Для этого используем параметр return_overflowing_tokens.
- return_offsets_mapping нужен, чтобы возвращать индексы начала и конца каждого токена в словарь в параметр offset_mapping, в нем для каждого токена его [l, r) индексы в исходном тексте

In [10]:
context = train_set[0]["context"]
question = train_set[0]["question"]

inputs : dict = tokenizer(question, context, max_length=32, truncation="only_second", stride=8, return_overflowing_tokens=True, return_offsets_mapping=True)

for ids in inputs["input_ids"]:
    print(tokenizer.decode(ids))

[CLS] чем представлены органические остатки? [SEP] В протерозойских отложениях органические остатки встречаются намного чаще, чем в ар [SEP]
[CLS] чем представлены органические остатки? [SEP] встречаются намного чаще, чем в архейских. Они представлены известковыми выделения [SEP]
[CLS] чем представлены органические остатки? [SEP] известковыми выделениями сине - зелёных водорослей, ход [SEP]
[CLS] чем представлены органические остатки? [SEP]елёных водорослей, ходами червей, остатками кишечнополос [SEP]
[CLS] чем представлены органические остатки? [SEP] остатками кишечнополостных. Кроме известковых водорослей, к [SEP]
[CLS] чем представлены органические остатки? [SEP]стковых водорослей, к числу древнейших растительных остатков относятся [SEP]
[CLS] чем представлены органические остатки? [SEP]ших растительных остатков относятся скопления графито - углистого вещества, образ [SEP]
[CLS] чем представлены органические остатки? [SEP] - углистого вещества, образовавшегося в результате разложени

In [11]:
len(inputs["input_ids"][0]) == len(inputs["offset_mapping"][0])

True

In [12]:
train_set.column_names

['id', 'title', 'context', 'question', 'answers']

In [13]:
train_set[0]

{'id': 62310,
 'title': 'SberChallenge',
 'context': 'В протерозойских отложениях органические остатки встречаются намного чаще, чем в архейских. Они представлены известковыми выделениями сине-зелёных водорослей, ходами червей, остатками кишечнополостных. Кроме известковых водорослей, к числу древнейших растительных остатков относятся скопления графито-углистого вещества, образовавшегося в результате разложения Corycium enigmaticum. В кремнистых сланцах железорудной формации Канады найдены нитевидные водоросли, грибные нити и формы, близкие современным кокколитофоридам. В железистых кварцитах Северной Америки и Сибири обнаружены железистые продукты жизнедеятельности бактерий.',
 'question': 'чем представлены органические остатки?',
 'answers': {'text': ['известковыми выделениями сине-зелёных водорослей'],
  'answer_start': [109]}}

Напишем функцию для препроцессинга:
1. Обрежем, западдим и токенизируем последовательности
2. Запишем в таргет индекс токена-начала и индекс токена-конца

In [14]:
train_set[0]["context"]

'В протерозойских отложениях органические остатки встречаются намного чаще, чем в архейских. Они представлены известковыми выделениями сине-зелёных водорослей, ходами червей, остатками кишечнополостных. Кроме известковых водорослей, к числу древнейших растительных остатков относятся скопления графито-углистого вещества, образовавшегося в результате разложения Corycium enigmaticum. В кремнистых сланцах железорудной формации Канады найдены нитевидные водоросли, грибные нити и формы, близкие современным кокколитофоридам. В железистых кварцитах Северной Америки и Сибири обнаружены железистые продукты жизнедеятельности бактерий.'

In [15]:
tokenized_samples = tokenizer(
    train_set[0]["question"],
    train_set[0]["context"],
    truncation="only_second",
    max_length=32,
    stride=8,
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
    padding="max_length",
)
print(tokenized_samples["attention_mask"][-1])
print(tokenized_samples["offset_mapping"][0])
tokenizer.decode(tokenized_samples["input_ids"][-1])

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]
[(0, 0), (0, 3), (4, 16), (17, 23), (23, 29), (30, 32), (32, 35), (35, 37), (37, 38), (0, 0), (0, 1), (2, 7), (7, 9), (9, 11), (11, 16), (17, 19), (19, 26), (26, 27), (28, 34), (34, 40), (41, 43), (43, 46), (46, 48), (49, 60), (61, 64), (64, 68), (69, 73), (73, 74), (75, 78), (79, 80), (81, 83), (0, 0)]


'[CLS] чем представлены органические остатки? [SEP]е продукты жизнедеятельности бактерий. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'

In [16]:
def prepare_train_data(examples: dict):  # в словарь маппится целый батч
    examples["question"] = [q.lstrip() for q in examples["question"]]

    tokenized_samples = tokenizer(
        examples["question"],
        examples["context"],
        truncation="only_second",
        max_length=MAX_LENGTH,
        stride=DOC_STRIDE,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )  # получает список текстов, возвращает список списков токенов
    # все передаваемые на вход списки склеивает, получая один список вопрос _+ ответ
    offset_mapping = tokenized_samples.pop("offset_mapping") # токен - [l, r) позиция его в начальном тексте
    overflow_to_sample_mapping = tokenized_samples.pop("overflow_to_sample_mapping")  # в случае переполнения поймем, к какому начальному тексту относится кусок 

    answer_start_positions = [] # Индексы токенов начала ответа
    answer_end_positions = [] # индексы токенов конца ответ
    masked_offset_mapping = []
    
    # examples["answers"][i]["answer_start"] - начало ответа для i-того текста
    # examples["answers"][i]["answer_start"] + len(examples["answers"][i]["text"]) - конец i-того текста не включительно
    # Если ответ поместился полностью, ставим
    for i, offsets in enumerate(offset_mapping):
        sample_index = overflow_to_sample_mapping[i]
        answer_start_index = examples["answers"][sample_index]["answer_start"][0]
        answer_end_index = answer_start_index + len(examples["answers"][sample_index]["text"][0])
        # ответ в [answer_start_index, answer_end_index]
        cls_token_index = tokenized_samples["input_ids"][sample_index].index(tokenizer.cls_token_id)  # на cls сгружаем все ненайденные ответы

        pointer = 0
        sequence_ids = tokenized_samples.sequence_ids(i) # так получаем список, который маппит склеенный текст в индексы переданных тексто
        masked_offset_mapping.append([offset if sequence_ids[i] == 1 else None for i, offset in enumerate(offsets)])
        
        while pointer < len(sequence_ids) and sequence_ids[pointer] != 1: 
            pointer += 1
        context_start = pointer
        
        while pointer < len(sequence_ids) and sequence_ids[pointer] == 1:
            pointer += 1
        context_end = pointer - 1
        
        if answer_end_index - 1 > offsets[context_end][1]: # ответ разорван и не входит в текст
            answer_start_positions.append(cls_token_index)
            answer_end_positions.append(cls_token_index)
        else:
            # ответ полностью входит в наш кусок
            # индексы в offsets стоят по каждому тексту отдельно. Нам нужно стартовать с позиции старта контекста.
            pointer = context_start
            while pointer <= context_end and offsets[pointer][0] <= answer_start_index: # берем последний токен, начало которого <= индекс старта
                # pointer-тый токен стоит в тексте на позиции [left_ind, right_ind)
                pointer += 1
            answer_start_positions.append(pointer - 1) # пушим индекс в склеенной последовательности токенов (в одном контексте это j - context_start + 1)
            
            pointer = context_end
            while pointer >= context_start and offsets[pointer][1] >= answer_end_index: # берем с запасом, чтобы токен точно вошел
                # pointer-тый токен стоит в тексте на позиции [left_ind, right_ind)
                pointer -= 1
            answer_end_positions.append(pointer + 1) # пушим индекс в склеенной последовательности токенов (в одном контексте это j - context_start + 1)
            
    tokenized_samples["start_positions"] = answer_start_positions
    tokenized_samples["end_positions"] = answer_end_positions
    tokenized_samples["offset_mapping"] = masked_offset_mapping
    
    return tokenized_samples

res = prepare_train_data(train_set[0:-1]) 
answer_start = res["start_positions"]
answer_end = res["end_positions"]
len(train_set), len(res), len(answer_start), len(answer_end)

(45328, 6, 62033, 62033)

remove_columns - удаляет колонки после выхода из функции

In [17]:
tokenized_train = train_set.map(prepare_train_data, batched=True, remove_columns=train_set.column_names)

In [18]:
tokenized_valid = valid_set.map(prepare_train_data, batched=True, remove_columns=valid_set.column_names)

In [19]:
def prepare_test_data(examples: dict):
    examples["question"] = [q.lstrip() for q in examples["question"]]

    tokenized_samples = tokenizer(
        examples["question"],
        examples["context"],
        truncation="only_second",
        max_length=MAX_LENGTH,
        stride=DOC_STRIDE,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    id_list = []
    masked_offset_mapping = []
    overflow_to_sample_mapping = tokenized_samples.pop("overflow_to_sample_mapping")  # в случае переполнения поймем, к какому начальному тексту относится кусок 
    
    for i in range(len(tokenized_samples["input_ids"])):
        sample_index = overflow_to_sample_mapping[i]  # добавляем поле айдишника
        id_list.append(examples["id"][sample_index])
        
        sequence_ids = tokenized_samples.sequence_ids(i)
        offsets = tokenized_samples["offset_mapping"][i]
        masked_offset_mapping.append([offset if sequence_ids[index] == 1 else None for index, offset in enumerate(offsets)])  # Маскируем все оффсеты не из контекста
    
    tokenized_samples["id"] = id_list
    tokenized_samples["offset_mapping"] = masked_offset_mapping
    return tokenized_samples

In [20]:
tokenized_test = test_set.map(prepare_test_data, batched=True, remove_columns=test_set.column_names)

Из колонок остаются поля с таргетом и сами данные

In [21]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

In [22]:
baseline = AutoModelForQuestionAnswering.from_pretrained(MODEL_CHECKPOINT)

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at cointegrated/rubert-tiny and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Ну пусть класс AutoModelForQuestionAnswering знает, как проводить обучение
Но откуда она знает, в каких полях датасета у меня лежит таргет, например?

In [23]:
model_name = MODEL_CHECKPOINT.split("/")[-1]
args = TrainingArguments(
    f"{model_name}",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=NUM_EPOCHS,
    weight_decay=0.01
)

In [24]:
from transformers import default_data_collator

data_collator = default_data_collator

In [25]:
trainer = Trainer(
    baseline,
    args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_valid,
    data_collator=data_collator,
    tokenizer=tokenizer
)

In [26]:
tokenized_train

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions', 'offset_mapping'],
    num_rows: 62035
})

Класс Trainer определяет поле для таргета по модели.
В датасете может быть записано что угодно, но главное, чтобы был весь набор ожидаемых ключей.
В нашем случае это таргет в start_positions и end_positions

Обучим / определим модель

In [27]:
from pathlib import Path

In [28]:
model_path = Path(f"./{model_name}")
if not model_path.exists():
    trainer.train()
    trainer.save_model(str(model_path))
    model = trainer.model
else:
    model = AutoModelForQuestionAnswering.from_pretrained(str(model_path))
    trainer.model = model

In [29]:
import torch

for batch in trainer.get_eval_dataloader():
    print("batch keys:", batch.keys())
    break
batch = {key: value.to(trainer.args.device) for key, value in batch.items()}
with torch.no_grad():
    output = model(**batch)  # Ждет те же данные, что и для обучения
print("output keys: ", output.keys())

batch keys: dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions'])
output keys:  odict_keys(['loss', 'start_logits', 'end_logits'])


In [37]:
def inference(data: dict | datasets.Dataset, n_best_size=20, max_answer_len=256, min_answer_len=0) -> str:
    """
    :param data: словарь формата теста (поля question, context и id)
    :param n_best_size: сколько лучших значений логитов используется для поиска ответа
    :param max_answer_len: не рассматриваем ответы с большей длиной даже если у них высокий скор
    :param min_answer_len: аналогично не рассматриваем меньшую длину
    :return: ответ на вопрос
    """
    if isinstance(data, datasets.Dataset):
        tokenized_data = data.map(prepare_test_data, batched=True, remove_columns=data.column_names)
    else:
        tokenized_data = prepare_test_data(data)  # TODO в нашей функции используется поле id, чтобы показать, к какому начально тексту пределывать предсказание. Нужно ненулевое любое.
    preds = model(
        input_ids=torch.tensor(tokenized_data["input_ids"], dtype=torch.int64),
        attention_mask=torch.tensor(tokenized_data["attention_mask"], dtype=torch.int64)
    )
    get_pieces_index = {} # словарь id существующего текста: индексы всех его кусков после токенизации
    for i, sample_index in enumerate(tokenized_data["id"]):
        if sample_index not in get_pieces_index:
            get_pieces_index[sample_index] = []
        get_pieces_index[sample_index].append(i)
    
    total_answers = []
    for sample_index, sample in enumerate(tqdm(data["id"])):
        context = data["context"][sample_index] # текущий контекст
        
        valid_answers = []  # ответы для одного контекста
        for piece_index in get_pieces_index[sample]: # перебираем все куски одного контекста, собираем ответы для каждого
            start_logits = preds.start_logits[piece_index].detach().numpy() 
            end_logits = preds.end_logits[piece_index].detach().numpy()
            offset_mapping = tokenized_data["offset_mapping"][piece_index]
            
            best_start_tokens_ids = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            best_end_tokens_ids = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            
            for start_token in best_start_tokens_ids:
                for end_token in best_end_tokens_ids:
                    if offset_mapping[start_token] is None or offset_mapping[end_token] is None:
                        continue
                    if (
                            start_token > end_token or 
                            end_token - start_token + 1 > max_answer_len or 
                            end_token - start_token + 1 < min_answer_len
                    ):
                        continue
                    valid_answers.append({
                            "score": start_logits[start_token].item() + end_logits[end_token].item(),
                            "text": context[offset_mapping[start_token][0] : offset_mapping[end_token][1]]
                        })
            if not valid_answers:
                valid_answers.append({"score": 0.0, "text": ""})
                
        total_answers.append(sorted(valid_answers, key=lambda x: -x["score"])) # для всего контекста берем лучший по всем кусочкам
    
    return total_answers
    
some_samples = test_set[list(range(3))]
inference(some_samples)

100%|██████████| 3/3 [00:00<00:00, 374.35it/s]


[[{'score': 5.860038161277771, 'text': 'ние'},
  {'score': 1.9342454671859741,
   'text': 'Многоклеточный организм — внесистематическая категория живых организмов, тело которых состоит из многих клеток, большая часть которых (кроме стволовых, например, клеток камбия у растений) дифференцированы, то есть различаются по строению и выполняемым функциям'},
  {'score': 1.8518097400665283,
   'text': 'внесистематическая категория живых организмов, тело которых состоит из многих клеток, большая часть которых (кроме стволовых, например, клеток камбия у растений) дифференцированы, то есть различаются по строению и выполняемым функциям'},
  {'score': 1.6291027665138245,
   'text': 'У колониальных организмов отсутствуют настоящие дифференцированные клетки, а следовательно, и разделение тела на ткани. Граница между многоклеточностью и колониальностью нечёткая'},
  {'score': 1.5599794536828995,
   'text': 'из многих клеток, большая часть которых (кроме стволовых, например, клеток камбия у растений)

In [38]:
inference({"question": ["Сколько Никите Ляпину годиков?"], "context": ["Никите Ляпину 20 лет"], "id": [0]})

100%|██████████| 1/1 [00:00<?, ?it/s]


[[{'score': 3.2535845041275024, 'text': 'Никите Ляпину 20 лет'},
  {'score': 2.3350441455841064, 'text': 'Н'},
  {'score': 1.7073487639427185, 'text': '20 лет'},
  {'score': 0.436442494392395, 'text': 'Ляпину 20 лет'}]]

Глазами видно, что модель назначает высочайшие скоры очень коротким ответам.
Возможно я допустил багу в препроцессинге, надо посмотреть, как распределена установленная мной длина ответа в токенах.
Возможно у нас есть проблемы с обучением.

Сначала оценим-ка метрику

In [43]:
import evaluate

metric = evaluate.load("squad")

In [ ]:
predictions = inference(valid_set[:500])
final_predictions = [sample[0]["text"] for sample in predictions]
formatted_predictions = [{"id": i, "prediction_text": text} for i, text in final_predictions.items()]
references = [{"id": i, "answers": sample["answers"]} for i, sample in enumerate(valid_set)]

In [ ]:
metric.compute(predictions=formatted_predictions, references=references)

Теперь напишем ручной цикл в торче
Plot Losses + warmup

Также в интерфейсе на инференсе можно выводить несколько дополнительных враиантов ответа
Типа ответами могут быть: ...

Также причесать код

In [ ]:
# from torch.utils.data import DataLoader
# from transformers import default_data_collator
# 
# train_set.set_format("torch")
# validation_set = valid_set.remove_columns(["example_id", "offset_mapping"])
# validation_set.set_format("torch")
# 
# train_dataloader = DataLoader(
#     train_set,
#     shuffle=True,
#     collate_fn=default_data_collator,
#     batch_size=8,
# )
# eval_dataloader = DataLoader(
#     validation_set, collate_fn=default_data_collator, batch_size=BATCH_SIZE
# )